In [ ]:
import qlib
from qlib.constant import REG_CN
qlib.init(region=REG_CN)

In [ ]:
from qlib.data import D
from qlib.contrib.data.handler import Alpha158
from qlib.workflow.record_temp import PortAnaRecord

In [ ]:
from qlib.workflow import R
exps = R.list_experiments()
exps

In [ ]:
exp = R.get_exp(experiment_name='train_model')
exp

In [ ]:
rid = list(exp.list_recorders().keys())[0]
rid

In [ ]:
recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
model = recorder.load_object("trained_model")

In [ ]:
from qlib.utils import exists_qlib_data, init_instance_by_config
market = "csi300"
data_handler_config = {
    "start_time": "2025-08-01",
    "end_time": "2025-08-30",
    "instruments": market,
}

segments = {
    "test": ("2025-08-01", "2025-08-30"),
}
dataset_config = {
    "class": "DatasetH",
    "module_path": "qlib.data.dataset",
    "kwargs": {
        "handler": {
            "class": "Alpha158",
            "module_path": "qlib.contrib.data.handler",
            "kwargs": data_handler_config,
        },
        "segments": segments,
    },
}
dataset = init_instance_by_config(dataset_config)

In [ ]:
dataset.handler.fetch(col_set=["feature"]).tail()

In [ ]:
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
with R.start(experiment_name="pridict"):
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

In [ ]:
pred_df = recorder.load_object("pred.pkl")
pred_df.index.names

In [ ]:
pred_df.loc['2025-08-29'].sort_values("score", ascending=False).head(10).index

In [ ]:
pred_df.xs('SH601688', level='instrument').plot()